In [1]:
import os
import openai
import sys

openai.api_key  = ""

In [2]:
from langchain.document_loaders import PyPDFLoader, WebBaseLoader
loaders = [
    PyPDFLoader("data/canon_printer_setup_instructions.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [4]:
docs[10]

Document(page_content='System Requirements\n23ES-009\nThe following system environment is required to install printer drivers.\nOS ●Windows Vista\n●Windows 7\n●Windows 8.1\n●Windows 10\n●Windows Server 2008\n●Windows Server 2008 R2 (64-bit version only)\n●Windows Server 2012 (64-bit version only)\n●Windows Server 2012 R2 (64-bit version only)\n●Windows Server 2016\nComputer A computer running one of the above operating systems\n \n●For information about support for the latest operating systems and service packs, visit the Canon website\n( http://www.canon.com/  ).System Requirements\n9', metadata={'source': 'data/canon_printer_setup_instructions.pdf', 'page': 10})

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 100
)

splits = text_splitter.split_documents(docs)
len(splits)

169

In [10]:
persist_directory = 'printerdb/chroma/'

In [11]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(api_key=openai.api_key)
from langchain.vectorstores import Chroma


print(embedding)
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

client=<openai.resources.embeddings.Embeddings object at 0x112e53350> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x112e646e0> model='text-embedding-ada-002' dimensions=None deployment='text-embedding-ada-002' openai_api_version='' openai_api_base=None openai_api_type='' openai_proxy='' embedding_ctx_length=8191 openai_api_key=SecretStr('**********') openai_organization=None allowed_special=set() disallowed_special='all' chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None


In [12]:
print(vectordb._collection.count())

169


In [13]:
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [14]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

print(memory)

return_messages=True memory_key='chat_history'


In [15]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, api_key=openai.api_key)

In [17]:
from langchain.chains import ConversationalRetrievalChain
retriever=vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [27]:
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='How far away is the sun?'), AIMessage(content="I don't know the answer to that question."), HumanMessage(content='How far away is canon?'), AIMessage(content="I don't know.")]), return_messages=True, memory_key='chat_history')

In [19]:
def question(ques: str) -> str:
    print(qa({"question": ques})['answer'])

In [21]:
question("How far away is canon?")

I don't know.


In [13]:
question("Summarize this document")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


This document is a manual that provides information on how to use the manual itself, including explanations of notations used, disclaimers, copyright information, and trademarks. It is designed to guide users on how to navigate and utilize the contents effectively.


In [14]:
question("Is there a specific model it's talking about?")

The document is referring to a Canon printer model, but the specific model number is not provided in the context provided.


In [15]:
question("Give me the steps to install it")

Based on the information provided in the document, the steps to install the Canon printer model are as follows:

1. Log on to the computer with an administrator account.
2. Download the printer driver from the Canon website ( http://www.canon.com/ ).
3. Unzip the downloaded file.
4. Connect the printer to the computer (wired LAN, wireless LAN, or USB connection).
5. If making a USB connection, turn OFF the printer before installing the driver.
6. Open the Printer Status Window folder if included in the downloaded file.
7. Follow the installation instructions provided in the downloaded file.

These steps should help you install the Canon printer model successfully.


In [16]:
question("Where's the website for Step 2 ?")

The website for downloading the printer driver in Step 2 is http://www.canon.com/.


In [17]:
#Page 62
question("I'm stuck at step 4. My printer is not showing up in the list. What do I do?")

If your printer is not showing up in the list during the installation process at step 4, you may be experiencing problems with the Wired LAN/Wireless LAN Connection. You should refer to page 62 of the manual for troubleshooting steps to address this issue. Additionally, you can select the [Set Printer Information] check box, set the printer information as necessary, and click [Next] to proceed with the installation.


In [18]:
#Page 64
question("I don't think my USB connection is working properly.")

Yes, there are troubleshooting steps provided for USB connection issues during installation of the Canon printer driver. Here are some steps to troubleshoot USB connection problems:

1. Ensure the printer is turned ON.
2. Check if the printer and computer are connected correctly via a USB cable.
3. If the printer was turned ON before installing the driver, turn OFF the printer, unplug the USB cable, and then reinstall the driver.
4. Exchange the USB cable, especially if it's a long one, for a shorter one.
5. If using a USB hub, connect the printer directly to the computer using a USB cable.
6. Delete the USB class driver and then reinstall the printer driver.

These steps should help troubleshoot USB connection issues during the installation of the Canon printer driver.


In [33]:
question("How does a shorter cable help?")

Using a shorter USB cable is recommended for troubleshooting USB connection issues during printer installation because longer USB cables can sometimes cause signal degradation or interference, leading to communication problems between the printer and the computer. By using a shorter USB cable, you can ensure a more stable and reliable connection, which can help resolve installation issues.


In [28]:
import pandas as pd
import tiktoken

from utils.embeddings_utils import get_embedding

ModuleNotFoundError: No module named 'utils'